In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar -xvf spark-3.2.1-bin-hadoop3.2.tgz
!java -version
!pip install findspark

In [2]:
import os 
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.types import *
from pyspark.sql.functions import col

## You can add more config while building 
spark = SparkSession.builder.master("local[8]").\
                    config("spark.app.name","session_one").\
                    getOrCreate() #number of threads = 16

In [ ]:
df = spark.read.csv("testpeople.csv",header=True,inferSchema=True)
df.show()

In [134]:
def listOfFrequencyTables(df): #take main dataframe, generate frequency dataframes
  histograms = []
  for col in df.dtypes:
      h=df.groupBy(col[0]).count()
      h = h.sort(desc("count"))
      histograms.append(h)
      h.show() #comment this line to suppress output
  return histograms
histograms = listOfFrequencyTables(df)

+----+-----+
|Name|count|
+----+-----+
|Nick|    4|
|Mary|    3|
|John|    3|
+----+-----+

+---------+-----+
|     city|count|
+---------+-----+
|  utrecht|    4|
|rotterdam|    4|
|amsterdam|    2|
+---------+-----+

+---+-----+
|age|count|
+---+-----+
| 21|    4|
| 22|    3|
| 20|    3|
+---+-----+



In [ ]:
def getDecompFromTopFrequencies(histograms):
  clusterlst=[]
  for i in range(len(histograms)): #query database with top values of all columns
    d= str(histograms[i].first()) #value of the first row
    print(d)
    d = d.split(",")[0].split('=')[1] #the splits are for formatting the string
    print(d)
    #print("d before:",d)
    if "'"  in d:
      d = d.split("'")[1]

    #print("d after:",d)
    #print(type(d))
    cname = str(histograms[i][0]).split("'")[1]
    print(cname,"=",d)
    
    data = (df.filter(col(cname) == d))
    
    data.show(15)
    clusterlst.append(data)
  return clusterlst

clusterlst = getDecompFromTopFrequencies(histograms)


In [220]:
from functools import reduce
from pyspark.sql.functions import lit
from pyspark.sql import DataFrame
def getDecompUsingFreqTable(df,freqdf): #takes original database and one frequency table as input, returns union of all queried dataframes as output
  print("this is frequency table:")
  freqdf.show()
  cname = freqdf.columns[0]
  valuelist = (freqdf.select(freqdf.columns[0]).rdd.flatMap(lambda x: x).collect()) #list of all values of frequency column
  #print("valuelist:",valuelist)
  #print("these are queries for the frequency table")
  unionlst = []
  for v in valuelist:
    result = df.filter(col(cname) == v)
    query = cname + "=" + v
    print(query)
    newres=result.withColumn("query",lit(query))
    #newres.show()
    unionlst.append(newres)
  unn = reduce(DataFrame.unionAll, unionlst) #put all queried dataframes back together as one
  print("union:")
  unn.show()

  return unn


union = (getDecompUsingFreqTable(df,histograms[0])) #function call with 'Name' frequency table
print("equality result:")
subdf = are_dfs_equal(df,union.drop('query')) #checks if union of queries covers whole database
print(subdf)


this is frequency table:
+----+-----+
|Name|count|
+----+-----+
|Nick|    4|
|Mary|    3|
|John|    3|
+----+-----+

Name=Nick
Name=Mary
Name=John
union:
+----+---------+---+---------+
|Name|     city|age|    query|
+----+---------+---+---------+
|Nick|rotterdam| 22|Name=Nick|
|Nick|  utrecht| 21|Name=Nick|
|Nick|  utrecht| 20|Name=Nick|
|Nick|amsterdam| 20|Name=Nick|
|Mary|amsterdam| 21|Name=Mary|
|Mary|rotterdam| 22|Name=Mary|
|Mary|  utrecht| 22|Name=Mary|
|John|  utrecht| 20|Name=John|
|John|rotterdam| 21|Name=John|
|John|rotterdam| 21|Name=John|
+----+---------+---+---------+

equality result:
dataframes are equal
True


In [213]:
def are_dfs_equal(df1, df2): #this works, i tested it
  res = df1.subtract(df2) #set subtraction on the two dataframes. 
  if res.count() == 0: #subtraction yielded empty set
    print("dataframes are equal")
    return True
  else:
    print("error! these rows are not in the union of your queries:")
    res.show() #show which tuples are not included in your query union
    return False

In [218]:
listOfFrequencyTables(clusterlst[0])

+----+-----+
|Name|count|
+----+-----+
|Nick|    4|
+----+-----+

+---------+-----+
|     city|count|
+---------+-----+
|  utrecht|    2|
|amsterdam|    1|
|rotterdam|    1|
+---------+-----+

+---+-----+
|age|count|
+---+-----+
| 20|    2|
| 22|    1|
| 21|    1|
+---+-----+



[DataFrame[Name: string, count: bigint],
 DataFrame[city: string, count: bigint],
 DataFrame[age: int, count: bigint]]

In [ ]:
#ignore this stuff lol

'''
  clusterlst=[]
  for i in range(len(df)): #query database with top values of all columns
    d= str(histograms[i].first()) #value of the first row
    print(d)
    d = d.split(",")[0].split('=')[1] #the splits are for formatting the string
    print(d)
    #print("d before:",d)
    if "'"  in d:
      d = d.split("'")[1]

    #print("d after:",d)
    #print(type(d))
    cname = str(histograms[i][0]).split("'")[1]
    print(cname,"=",d)
    
    data = (df.filter(col(cname) == d))
    
    data.show(15)
    clusterlst.append(data)
  return clusterlst

clusterlst = getDecompFromTopFrequencies(histograms)
'''
'''
for i in range(len(histograms)):
  d= str(histograms[i].first()) #first row
  d = d.split(",")[0].split('=')[1]
  if "'"  in d:
    d = d.split("'")[1]
  cname = str(histograms[i][0]).split("'")[1] #column name
  data = filterdf(df,cname,d)
  data.show()
'''